In [1]:
"""
4 / 5 (순서 / 메모리 종류)
ConversationSummaryBufferMemory
ConversationSummaryMemory 와 ConversationBufferMemory의 결함
최근의 대화를 max_token_limit의 수만큼 저장하고, 그 전의 대화들은 요약해서 저장한다.
장점: 
단점: 

"""

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})

def get_history():
  return memory.load_memory_variables({})

add_message("Hi I'm bigPerson, I live in South Korea", "Wow that is so cool!")


In [2]:
get_history()

{'history': [HumanMessage(content="Hi I'm bigPerson, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [3]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [4]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as bigPerson and mentions they live in South Korea.'),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [5]:
add_message("How far is Korea from Argentina?", "I don't know well. Super far!")

In [6]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as bigPerson and mentions they live in South Korea. The AI responds with enthusiasm, saying "Wow, that is so cool!"'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know well. Super far!")]}